In [120]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk as nl
import tensorflow as tf
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import regex as re
stop_word = set(stopwords.words('english'))
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-data/spam.csv


In [121]:
data=pd.read_csv('/kaggle/input/spam-data/spam.csv', encoding='latin')

In [122]:
data.head()
data1=data.iloc[:,:2]

In [123]:
def removing_punctuation(val):
    val2="".join([v for v in val if v not in string.punctuation])
    val1=val2.lower()
    return val2
def stop_words(val):
    val2=[v for v in val if v not in stop_word]
    return val2

In [124]:
data1['remove_punctuation']=data1['v2'].apply(lambda x:removing_punctuation(x))
data1.head()

/tmp/ipykernel_32/1104584760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['remove_punctuation']=data1['v2'].apply(lambda x:removing_punctuation(x))


,v1,v2,remove_punctuation
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...


In [125]:
data1['lower_case']=data1['remove_punctuation'].apply(lambda x:x.lower())
data1.head()

/tmp/ipykernel_32/201798011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['lower_case']=data1['remove_punctuation'].apply(lambda x:x.lower())


,v1,v2,remove_punctuation,lower_case
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,nah i dont think he goes to usf he lives aroun...


In [126]:
data1['tokenization']=data1['lower_case'].apply(lambda x:word_tokenize(x))

In [127]:
data1['remove_stop_words']=data1['tokenization'].apply(lambda x:stop_words(x))
data1['remove_stop_words']

0       [go, jurong, point, crazy, available, bugis, n...
1                          [ok, lar, joking, wif, u, oni]
2       [free, entry, 2, wkly, comp, win, fa, cup, fin...
3           [u, dun, say, early, hor, u, c, already, say]
4       [nah, dont, think, goes, usf, lives, around, t...
                              ...                        
5567    [2nd, time, tried, 2, contact, u, u, å£750, po...
5568                   [ì, b, going, esplanade, fr, home]
5569                     [pity, mood, soany, suggestions]
5570    [guy, bitching, acted, like, id, interested, b...
5571                                   [rofl, true, name]
Name: remove_stop_words, Length: 5572, dtype: object

In [128]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

In [129]:
def stemmer(val):
    val2=[porter_stemmer.stem(v) for v in val]
    return val2

In [130]:
data1['stemming']=data1['remove_stop_words'].apply(lambda x:stemmer(x))

In [131]:
from sklearn.feature_extraction.text import CountVectorizer

In [132]:
countV = CountVectorizer()
transformed = countV.fit_transform(data1['lower_case'].values)

In [133]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(transformed, data['v1'], test_size = 0.20)

In [134]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=300, n_jobs=2)
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=300, n_jobs=2)

In [135]:
y_pred = rf_classifier.predict(x_test)

In [136]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
data1['v1']= data1['v1'].replace({'ham': 0, 'spam': 1})

              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       956
        spam       1.00      0.81      0.90       159

    accuracy                           0.97      1115
   macro avg       0.98      0.91      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [145]:
from tensorflow.keras.preprocessing.text import Tokenizer

token=Tokenizer(oov_token='<oov>')
words=token.fit_on_texts(data1['lower_case'])
seq=token.texts_to_sequences(data1['lower_case'])
pad=tf.keras.preprocessing.sequence.pad_sequences(seq)
max_len= len(pad[0])
vocab_len=len(token.word_index)+1

In [170]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_len,5),
    tf.keras.layers.GlobalAvgPool1D(),
    tf.keras.layers.Dense(24,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [171]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [172]:
x_train, x_test, y_train, y_test = pad[:700],pad[700:],data1['v1'][:700],data1['v1'][700:]
y_train


0      0
1      0
2      1
3      0
4      0
      ..
695    0
696    0
697    0
698    0
699    0
Name: v1, Length: 700, dtype: int64

In [173]:
spam=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test))

Epoch 1/50
22/22 [==============================] - 1s 27ms/step - loss: 0.6742 - accuracy: 0.8571 - val_loss: 0.6563 - val_accuracy: 0.8672
Epoch 2/50
22/22 [==============================] - 0s 19ms/step - loss: 0.6410 - accuracy: 0.8571 - val_loss: 0.6200 - val_accuracy: 0.8672
Epoch 3/50
22/22 [==============================] - 0s 17ms/step - loss: 0.6030 - accuracy: 0.8571 - val_loss: 0.5771 - val_accuracy: 0.8672
Epoch 4/50
22/22 [==============================] - 0s 19ms/step - loss: 0.5588 - accuracy: 0.8571 - val_loss: 0.5287 - val_accuracy: 0.8672
Epoch 5/50
22/22 [==============================] - 0s 17ms/step - loss: 0.5115 - accuracy: 0.8571 - val_loss: 0.4805 - val_accuracy: 0.8672
Epoch 6/50
22/22 [==============================] - 0s 20ms/step - loss: 0.4671 - accuracy: 0.8571 - val_loss: 0.4392 - val_accuracy: 0.8672
Epoch 7/50
22/22 [==============================] - 0s 19ms/step - loss: 0.4329 - accuracy: 0.8571 - val_loss: 0.4093 - val_accuracy: 0.8672
Epoch 8/50
22

700     0
701     0
702     0
703     0
704     0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: v1, Length: 4872, dtype: int64

In [181]:
from sklearn.metrics import precision_score
y_pred=model.predict(x_test)

# precision = precision_score(y_test,y_pred)

153/153 [==============================] - 0s 2ms/step


In [182]:
threshold = 0.5
binary_predictions = (y_pred > threshold).astype(int)

In [185]:
binary_predictions
precision = precision_score(y_test,binary_predictions)
precision

1.0